# Dogs vs. Cats
This is my first Kaggle competition and also my first deep learning project from scratch.  
This notebook is for implementing training and testing process using pytorch.
![spring](https://kaggle2.blob.core.windows.net/competitions/kaggle/3362/media/woof_meow.jpg)

## Preprocessing Data
### 1.  Data loading and transforming

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets 
import numpy as np
import os


In [2]:
batch_size = 64
modes = ['train', 'val']
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224,scale=(0.08,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    
}

dogcat_datasets = {mode: datasets.ImageFolder(mode, data_transforms[mode]) for mode in modes}
dogcat_dataloader = {mode: DataLoader(dogcat_datasets[mode], batch_size=batch_size,
                                      shuffle=True, num_workers=0)
                     for mode in modes}

class_names = dogcat_datasets['train'].classes
num_classes = len(class_names)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### 2. Visualizing data

In [ ]:
def show_imgs(images, labels):
    for i in range(len(images)):
        image = images[i].numpy().transpose(1,2,0)
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        image = np.clip(image*std+mean, 0, 1)
        ax = plt.subplot(len(images)//8,8,i+1)
        ax.set_title(class_names[labels[i]])
        ax.axis('off')
        plt.imshow(image)

batch_1st = iter(dogcat_dataloader['train']).next()
fig = plt.figure(figsize=(15,15))
print(f'imgs batch size: {batch_1st[0].size()}, labels batch size: {batch_1st[1].size()}')
print(f'Images in 1st batch of training set:')
show_imgs(*batch_1st)

## Model and Training
### 3. Loading model

In [3]:
from model import *

# define a pretrained network Resnet-18
resnet = models.resnet18(pretrained=True)
model_res = Res(resnet, num_classes)
# define a network similar to darknet-19
model_dark = Darknet(num_classes)

### 4. Training and visualizing histories

In [ ]:
from train_and_test import *

criterion_res = nn.CrossEntropyLoss()
optimizer_res = optim.SGD(model_res.params, lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_pattern_res = optim.lr_scheduler.StepLR(optimizer_res, 10, gamma=0.1)
tracker_res = Epoch_History()

# 4a) transfer learning using Resnet, freeze all network except final layer
# History_tl = train_model(model_res, dogcat_dataloader, criterion_res, optimizer_res, 
#             num_epochs=1,
#             lr_pattern=lr_pattern_res, 
#             device=device, 
#             history_tracker=tracker_res)

In [ ]:
# define a small subset of train and val set to test train_model() function
from train_and_test import *
from torch.utils.data import Subset

sub_dogcat = {mode: Subset(dogcat_datasets[mode], range(100)) for mode in modes}
sub_dogcat_loader = {mode: DataLoader(sub_dogcat[mode], batch_size=10,
                                      shuffle=True, num_workers=0)
                     for mode in modes}

sub_H = train_model(model_res, sub_dogcat_loader, criterion_res, optimizer_res, 
                    lr_pattern=lr_pattern_res, 
                    device=device, 
                    num_epochs=2, 
                    history_tracker=tracker_res)

sub_H.plot_history()

In [ ]:
# 4b) training a self-defined model from scratch, construct similar to darknet19


# Testing and post processing predictions
### 5. Loading testset and testing

In [4]:
from test_dataset import *

# 5a) testing data without fivecrop
testset_nofivecrop = TestDataset(fivecrop=False, test_dir='./test/')

In [ ]:
# try test_model() function using a subset of total testset
from train_and_test import *
from torch.utils.data import Subset

sub_nofive = Subset(testset_nofivecrop, range(50))
sub_nofive_loader = DataLoader(sub_nofive)
r_nofive = test_model(model_res, sub_nofive_loader, device=device)

for r in r_nofive:
    print(r)

In [ ]:
# 5b) testing data with fivecrop
testset_fivecrop = TestDataset(fivecrop=True, test_dir='./test/')

In [ ]:
# try test_model() function using a subset of total testset
from train_and_test import *
from torch.utils.data import Subset

sub_five = Subset(testset_fivecrop, range(50))
sub_five_loader = DataLoader(sub_five)
r_five = test_model(model_res, sub_five_loader, fivecrop=True, device=device)

for r in r_five:
    print(r)

In [5]:
model_res.load_state_dict(torch.load('./weights_res.pt', map_location='cpu'))

In [6]:
from train_and_test_probability import *
from torch.utils.data import Subset

sub_nofive = Subset(testset_nofivecrop, range(100))
sub_nofive_loader = DataLoader(sub_nofive)
r_nofive = test_model(model_res, sub_nofive_loader, device=device)

------------------------------
Test finished, submission csv saved!
Images tested:    100
Total time:       23.672s
Time per image:   0.237s


In [7]:
from math import log
def logloss(n, y):
    if y==1:
        return -y*log(n)
    else:
        return -(1-y)*log(1-n)

loss = 0.0
for i, r in enumerate(r_nofive):
    if r[1]>0.5:
        y=1
    else:
        y=0
#     print(logloss(r[1], y))
    loss += logloss(r[1], y)
    loss /= (1+i)
print(f'{loss*12500/(i+1):.8f}', 1+i)

0.00002942 100
